In [1]:
#添加basic特征
import json
with open("mp_elastic_with_desc.json") as f:
    data = json.load(f)

In [2]:
for k, v in data[2].items():
    print(k, v)

builder_meta {'emmet_version': '0.71.1', 'pymatgen_version': '2023.10.4', 'pull_request': 990, 'database_version': '2023.11.1', 'build_date': '2023-10-20 07:05:00.912000', 'license': None}
nsites 3
elements ['Nb', 'Pd']
nelements 2
composition {'Nb': 1.0, 'Pd': 2.0}
composition_reduced {'Nb': 1.0, 'Pd': 2.0}
formula_pretty NbPd2
formula_anonymous AB2
chemsys Nb-Pd
volume 48.113747937927926
density 10.552156716548524
density_atomic 16.037915979309307
symmetry {'crystal_system': 'Orthorhombic', 'symbol': 'Immm', 'number': 71, 'point_group': 'mmm', 'symprec': 0.1, 'version': '1.16.5'}
property_name elasticity
material_id mp-1018151
deprecated False
deprecation_reasons None
last_updated 2023-10-20 07:05:00.913000
origins []
warnings []
structure {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0.0, 'lattice': {'matrix': [[-1.42880075, 1.98263577, 4.24614275], [1.42880075, -1.98263577, 4.24614275], [1.42880075, 1.98263577, -4.24614275]], 'pbc': [True, True, True], 'a'

In [3]:
def construct_datapoint(raw_datapoint: dict):
    dic = {}
    dic['crystal system'] : str = raw_datapoint['symmetry']['crystal_system']
    dic['description'] : str = raw_datapoint['description']
    # Extract the elastic tensor
    dic['elastic tensor'] : list or None = raw_datapoint['elastic_tensor']['ieee_format'] if raw_datapoint['elastic_tensor']['ieee_format'] else None
    # Traverse the 6x6 2D array elastic_tensor and change all -0.0 to 0.0
    if dic['elastic tensor']:
        for i in range(6):
            for j in range(6):
                if dic['elastic tensor'][i][j] == -0.0:
                    dic['elastic tensor'][i][j] = 0.0
    # Extract other elasticity properties
    dic['bulk modulus'] : dict or None = raw_datapoint['bulk_modulus'] if raw_datapoint['bulk_modulus'] else None
    dic['shear modulus'] : dict or None = raw_datapoint['shear_modulus'] if raw_datapoint['shear_modulus'] else None
    dic['young modulus'] : float or None = raw_datapoint['young_modulus'] if raw_datapoint['young_modulus'] else None
    dic['universal anisotropy'] : float or None = raw_datapoint['universal_anisotropy'] if raw_datapoint['universal_anisotropy'] else None
    dic['isotropic possion ratio'] : float or None = raw_datapoint['homogeneous_poisson'] if raw_datapoint['homogeneous_poisson'] else None

    return dic

In [4]:
def construct_dataset(raw_data):
    dataset = [construct_datapoint(data) for data in raw_data]
    print(f"Dataset constructed with {len(dataset)} data points")
    return dataset

def store_dataset(dataset, stored_file_name):
    with open(stored_file_name, 'w') as f:
        json.dump(dataset, f, indent=2)
    return print(f"Dataset stored in {stored_file_name}")

In [5]:
combined_dataset = construct_dataset(data)
store_dataset(combined_dataset, 'only_structure_desc.json')

Dataset constructed with 12127 data points
Dataset stored in only_structure_desc.json


In [6]:
#build_alpaca
import json

with open('only_structure_desc.json') as f:
    origin_data = json.load(f)

In [7]:
from pymatgen.analysis.elasticity import ElasticTensor
import numpy as np

print("Original data length", len(origin_data))
id_list = []
for i in range(len(origin_data)):
    try:
        elastic_constant = np.asarray(origin_data[i]['elastic tensor'])
        elastic_tensor = ElasticTensor.from_voigt(elastic_constant)
        y_m = round(elastic_tensor.y_mod / 1e9, 3)
    except Exception as e:
        id_list.append(i)

print(id_list)
print(len(id_list))

for index in sorted(id_list, reverse=True):
    del origin_data[index]

print("After removing data length", len(origin_data))

def calculate_properties(stiffness_matrix: list) -> (float, float, float):
    elastic_constant = np.asarray(stiffness_matrix)
    elastic_tensor = ElasticTensor.from_voigt(elastic_constant)
    youngs_modulus = round(elastic_tensor.y_mod / 1e9, 3)
    return youngs_modulus

#删除体积模量和剪切模量异常的数据
print("Original data length", len(origin_data))
delete_index = []
for i in range(len(origin_data)):
    if origin_data[i]['bulk modulus'] == None or origin_data[i]['shear modulus'] == None:
        delete_index.append(i)
    elif any(value < 0 for value in origin_data[i]['bulk modulus'].values()) or any(value > 1000 for value in origin_data[i]['bulk modulus'].values()):
        delete_index.append(i)
    elif any(value < 0 for value in origin_data[i]['shear modulus'].values()) or any(value > 1000 for value in origin_data[i]['shear modulus'].values()):
        delete_index.append(i)

for index in sorted(delete_index, reverse=True):
    del origin_data[index]

print("Deleted data with bulk_modulus or shear_modulus abnormality", len(origin_data))

Original data length 12127


/Users/siyuliu/anaconda3/envs/ec/lib/python3.10/site-packages/pymatgen/analysis/elasticity/elastic.py:172: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 / self.compliance_tensor.voigt[:3, :3].sum()
/Users/siyuliu/anaconda3/envs/ec/lib/python3.10/site-packages/pymatgen/analysis/elasticity/elastic.py:199: RuntimeWarning: invalid value encountered in scalar divide
  return 9.0e9 * self.k_vrh * self.g_vrh / (3 * self.k_vrh + self.g_vrh)
/Users/siyuliu/anaconda3/envs/ec/lib/python3.10/site-packages/pymatgen/analysis/elasticity/elastic.py:177: RuntimeWarning: divide by zero encountered in scalar divide
  return 15 / (


[53, 74, 159, 247, 338, 450, 488, 542, 552, 690, 719, 788, 901, 1044, 1079, 1097, 1154, 1210, 1216, 1229, 1302, 1381, 1399, 1466, 1477, 1520, 1529, 1556, 1564, 1604, 1629, 1649, 1717, 1822, 1920, 1925, 1941, 2025, 2195, 2222, 2232, 2257, 2521, 2571, 2590, 2676, 2692, 2818, 2831, 2893, 2946, 2980, 2985, 3026, 3045, 3060, 3124, 3210, 3215, 3221, 3230, 3251, 3265, 3273, 3352, 3423, 3451, 3504, 3568, 3626, 3739, 3787, 3791, 3792, 3859, 3935, 3988, 4013, 4024, 4032, 4039, 4169, 4174, 4211, 4255, 4304, 4305, 4436, 4507, 4516, 4534, 4579, 4600, 4607, 4648, 4652, 4740, 4767, 4912, 4981, 5005, 5051, 5061, 5142, 5337, 5340, 5431, 5488, 5525, 5553, 5626, 5653, 5715, 5745, 5798, 5855, 6025, 6027, 6091, 6113, 6153, 6172, 6251, 6355, 6416, 6423, 6454, 6457, 6470, 6510, 6516, 6534, 6582, 6595, 6615, 6619, 6674, 6705, 6718, 6746, 6747, 6750, 6758, 6767, 6771, 6791, 6860, 6885, 6893, 6943, 6944, 6990, 7035, 7189, 7197, 7198, 7199, 7208, 7210, 7272, 7282, 7365, 7401, 7407, 7557, 7560, 7574, 7633, 7691, 

In [8]:
after_deleted_data = origin_data
bulk_modulus = [item['bulk modulus']['vrh'] for item in after_deleted_data]
shear_modulus = [item['shear modulus']['vrh'] for item in after_deleted_data]
young_modulus = [calculate_properties(item['elastic tensor']) for item in after_deleted_data]

for i in range(len(after_deleted_data)):
    after_deleted_data[i]['young modulus'] = young_modulus[i]

In [10]:
import json
data = after_deleted_data

import math
crystal_structure_data = [item['crystal system'] for item in data]
crystal_structure_set = set(crystal_structure_data)
crystal_structure_count = len(crystal_structure_set)
crystal_structure_values_count = {structure: crystal_structure_data.count(structure) for structure in crystal_structure_set}
print(f'Crystal structures: {crystal_structure_count}')
print(f'Crystal structure values count: {crystal_structure_values_count}')
csv_p5 = {k: math.floor(v*0.05) for k, v in crystal_structure_values_count.items() if v >= 5}
print(f'5% for each crystal_system: {csv_p5}')

for k, v in csv_p5.items():
    for item in data:
        if item['crystal system'] == k:
            item['label'] = 'test'
            v -= 1
            if v == 0:
                break

for item in data:
    if 'label' not in item:
        item['label'] = 'train'

alpaca_train = []
alpaca_test = []
def build_alpaca(datapoint):
    dic = dict()
    dic['instruction'] = f"{datapoint}"
    dic['input'] = ""
    dic['output'] = ""
    return dic

for item in data:
    if item['label'] == 'train':
        alpaca_train.append(build_alpaca(item))
        del item['label']
    else:
        alpaca_test.append(build_alpaca(item))
        del item['label']

Crystal structures: 7
Crystal structure values count: {'Orthorhombic': 1386, 'Cubic': 4175, 'Monoclinic': 572, 'Tetragonal': 2036, 'Trigonal': 814, 'Triclinic': 66, 'Hexagonal': 1471}
5% for each crystal_system: {'Orthorhombic': 69, 'Cubic': 208, 'Monoclinic': 28, 'Tetragonal': 101, 'Trigonal': 40, 'Triclinic': 3, 'Hexagonal': 73}


In [11]:
for k, v in csv_p5.items():
    for item in data:
        if item['crystal system'] == k:
            item['label'] = 'test'
            v -= 1
            if v == 0:
                break

for item in data:
    if 'label' not in item:
        item['label'] = 'train'

In [12]:
for k, v in data[0].items():
    print(k, v)

crystal system Orthorhombic
description FeB crystallizes in the orthorhombic Cmcm space group. Fe(1)3+ is bonded in a 7-coordinate geometry to seven equivalent B(1)3- atoms. There are a spread of Fe(1)-B(1) bond distances ranging from 2.14-2.17 Å. B(1)3- is bonded in a 9-coordinate geometry to seven equivalent Fe(1)3+ and two equivalent B(1)3- atoms. Both B(1)-B(1) bond lengths are 1.80 Å.
elastic tensor [[450.0, 195.0, 171.0, 0.0, 0.0, 0.0], [195.0, 470.0, 150.0, 0.0, 0.0, 0.0], [171.0, 150.0, 345.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 177.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 203.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 187.0]]
bulk modulus {'voigt': 254.969, 'reuss': 247.879, 'vrh': 251.424}
shear modulus {'voigt': 163.419, 'reuss': 154.716, 'vrh': 159.067}
young modulus 393.981
universal anisotropy 0.31
isotropic possion ratio 0.239
label test


In [16]:
#构建alpaca数据集
def build_alpaca_dataset(data):
    train_dataset = []
    test_dataset = []
    instruction = "Given a material's crystal structure description, predict the elastic tensor of it directly and accurately with scientific logic. Answer without any other comments, descriptions, or explanations. The answer should be a 6x6 Python matrix. "
    for item in data:
        label = item['label']
        input_item = item['description']
        output_item = f"{item['elastic tensor']}"
        if label == 'test':
            test_dataset.append({
                'instruction': instruction,
                'input': input_item,
                'output': output_item,
            })
        else:
            train_dataset.append({
                'instruction': instruction,
                'input': input_item,
                'output': output_item,
            })
    return train_dataset, test_dataset

In [18]:
tr, te = build_alpaca_dataset(data)

with open('only_structure_desc_train.json', 'w') as file:
    json.dump(tr, file)

with open('only_structure_desc_test.json', 'w') as file:
    json.dump(te, file)

In [19]:
print(len(tr), len(te))
print(tr[0])
print(te[0])

9998 522
{'instruction': "Given a material's crystal structure description, predict the elastic tensor of it directly and accurately with scientific logic. Answer without any other comments, descriptions, or explanations. The answer should be a 6x6 Python matrix. ", 'input': 'TiB2 is hexagonal omega structure structured and crystallizes in the hexagonal P6/mmm space group. Ti(1) is bonded to twelve equivalent B(1) atoms to form a mixture of face and edge-sharing TiB12 cuboctahedra. All Ti(1)-B(1) bond lengths are 2.38 Å. B(1) is bonded in a 9-coordinate geometry to six equivalent Ti(1) and three equivalent B(1) atoms. All B(1)-B(1) bond lengths are 1.75 Å.', 'output': '[[652.0, 65.0, 106.0, 0.0, 0.0, 0.0], [65.0, 652.0, 106.0, 0.0, 0.0, 0.0], [106.0, 106.0, 451.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 259.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 259.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 293.0]]'}
{'instruction': "Given a material's crystal structure description, predict the elastic tensor of it direc